In [1]:
!pip install kaggle

In [2]:
!pip install opendatasets

In [3]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/kazanova/sentiment140")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: issac11
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140


100%|██████████| 80.9M/80.9M [00:00<00:00, 145MB/s]


In [4]:
import numpy as np
import pandas as pd
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
#printing the stopwords in English
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
#Loading the data from CSV file to pandas dataframe
twitter = pd.read_csv("/content/sentiment140/training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1")

In [8]:
#Checking the number of rows and columns
twitter.shape

(1599999, 6)

In [9]:
#printing the first 5 rows of the dataframe
twitter.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [10]:
#naming the columns and reading the dataset again
twitter.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']
twitter.head()

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
#counting the number of missing values in the dataset
twitter.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [12]:
#Checking the distribution of target columns
twitter['target'].value_counts()

,count
target,
4,800000
0,799999


**CONVERT THE TARGET "4" TO "1"**

In [13]:
twitter.replace({'target': {4: 1}}, inplace=True)
twitter['target'].value_counts()

,count
target,
1,800000
0,799999


0 --> NEGATIVE TWEET

1 --> POSITIVE TWEET

**STREAMING**

In [14]:
port_steam = PorterStemmer()

In [15]:
def streaming(content):
  content = re.sub('[^a-zA-Z]', ' ', content)
  content = content.lower()
  content = content.split()
  content = [port_steam.stem(word) for word in content if not word in stopwords.words('english')]
  content = ' '.join(content)
  return content

In [16]:
#-- MINUTES TO COMPLETE THIS EXICUTION
from tqdm import tqdm
tqdm.pandas()

twitter['text'] = twitter['text'].progress_apply(streaming)

100%|██████████| 1599999/1599999 [40:50<00:00, 653.00it/s]


In [17]:
twitter.head()

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset updat facebook text might cri result sch...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,kenichan dive mani time ball manag save rest g...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole bodi feel itchi like fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,nationwideclass behav mad see
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,kwesidei whole crew


In [18]:
print(twitter.columns)

Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')


In [19]:
print(twitter['text'])

0          upset updat facebook text might cri result sch...
1          kenichan dive mani time ball manag save rest g...
2                            whole bodi feel itchi like fire
3                              nationwideclass behav mad see
4                                        kwesidei whole crew
                                 ...                        
1599994                           woke school best feel ever
1599995    thewdb com cool hear old walt interview http b...
1599996                         readi mojo makeov ask detail
1599997    happi th birthday boo alll time tupac amaru sh...
1599998    happi charitytuesday thenspcc sparkschar speak...
Name: text, Length: 1599999, dtype: object


In [20]:
print(twitter['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599994    1
1599995    1
1599996    1
1599997    1
1599998    1
Name: target, Length: 1599999, dtype: int64


In [21]:
#Separating the data and label
x = twitter['text'].values
y = twitter['target'].values

In [22]:
print(x)

['upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound'
 'whole bodi feel itchi like fire' ... 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [23]:
print(y)

[0 0 0 ... 1 1 1]


**SPLITING THE DATA TRAINING DATA AND TEST DATA**

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

In [25]:
print(x.shape, x_train, x_test.shape)

(1599999,) ['watch saw iv drink lil wine'
 'hire anoth employe gourmet point current hurri'
 'punish know much work tomorrow everyon els get day' ...
 'ohjustjak awkward crush nph amazingg'
 'oliveandfig wow never tweet nah yet tuesday spoil'
 'girl jenn stay twitter b c got back late amp tri relax enough sleep work amp awak'] (320000,)


In [26]:
print(x_train)

['watch saw iv drink lil wine'
 'hire anoth employe gourmet point current hurri'
 'punish know much work tomorrow everyon els get day' ...
 'ohjustjak awkward crush nph amazingg'
 'oliveandfig wow never tweet nah yet tuesday spoil'
 'girl jenn stay twitter b c got back late amp tri relax enough sleep work amp awak']


In [27]:
print(x_test)

['school someon slap acrosss face caus stay class total dragggg poor poor kitti'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'dad comput turn even though weird mean download song want' ...
 'destini nevertheless hooray member wonder safe trip'
 'strawberri heavi appar broke blender epic fuck fail' 'supersandro thank']


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# Apply vectorizer to training and test data
x_train_tfidf = vectorizer.fit_transform(x_train)  # Store the transformed training data
x_test_tfidf = vectorizer.transform(x_test)        # Store the transformed test data


In [31]:
print(x_train_tfidf)

  (0, 436655)	0.27247657460069113
  (0, 354566)	0.358082129508793
  (0, 185125)	0.5296934860731303
  (0, 109298)	0.37470892870977707
  (0, 234920)	0.4196620311217523
  (0, 443035)	0.44770025018080273
  (1, 166826)	0.3977368489124923
  (1, 19762)	0.25286013616648123
  (1, 119689)	0.4309123985667578
  (1, 152015)	0.49196931362728563
  (1, 319721)	0.3128279791558143
  (1, 86730)	0.33015139495167095
  (1, 172343)	0.3785321144028949
  (2, 326288)	0.5853283798404598
  (2, 220710)	0.2626149875630722
  (2, 279017)	0.28499749936226054
  (2, 445635)	0.2356370052189603
  (2, 412554)	0.28961919890983545
  (2, 124728)	0.3317593891539697
  (2, 117914)	0.39861155079647814
  (2, 146003)	0.22130486737209085
  (2, 93826)	0.22330494345030885
  (3, 93826)	0.46202690964703447
  (3, 331805)	0.4656145237310222
  (3, 46285)	0.6887696820070169
  :	:
  (1279996, 13878)	0.4358882391561783
  (1279996, 295620)	0.47908235635831214
  (1279996, 299643)	0.5685438365651838
  (1279997, 379348)	0.39076222969980035
  (127

In [32]:
print(x_test_tfidf)

  (0, 63768)	0.26422682589785035
  (0, 75261)	0.26552138110472423
  (0, 127038)	0.2730995033465573
  (0, 219133)	0.3295455088887744
  (0, 320624)	0.5238287544743887
  (0, 355868)	0.22363572756759686
  (0, 371332)	0.3972681545090477
  (0, 375613)	0.24658637821166524
  (0, 382367)	0.2541115193458825
  (0, 414033)	0.2624690535388268
  (1, 6452)	0.3061987253241848
  (1, 15098)	0.21044041574811612
  (1, 145408)	0.5774478727480551
  (1, 217431)	0.40260807391422926
  (1, 256686)	0.2861349119561766
  (1, 348128)	0.4737202797635419
  (1, 366259)	0.24522188688615762
  (2, 79341)	0.34512546063620464
  (2, 88422)	0.33529803711277506
  (2, 107866)	0.36857211925047617
  (2, 124443)	0.2729825984376057
  (2, 258609)	0.3072614985829275
  (2, 375890)	0.3020734075258265
  (2, 407333)	0.27060796399382786
  (2, 418804)	0.33257344872506
  :	:
  (319995, 193560)	0.373116119782695
  (319995, 307063)	0.3842384782822585
  (319995, 309043)	0.23209118010710333
  (319995, 345420)	0.3312796902454122
  (319995, 4421

TRAINING THE MACHINE LEARNING MODEL

LOGISTIC REGRESSION

In [33]:
model = LogisticRegression(max_iter=1000)

In [34]:
model.fit(x_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

In [36]:
model.score(x_train_tfidf, y_train)

0.801063125830567

MODEL EVALUATION

ACCURACY SCORE

In [37]:
#Accuracy Score on the training data
x_train_prediction = model.predict(x_train_tfidf)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [38]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.801063125830567


In [39]:
#Accuracy Score on the test data
x_test_prediction = model.predict(x_test_tfidf)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [40]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.7771625


**MODEL ACCURACY = 77.7%**

Saving the training model

In [41]:
import pickle

In [42]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using the Saved model for Future Predictions

In [43]:
#loading the saved model
Loded_model = pickle.load(open('trained_model.sav', 'rb'))

In [50]:
x_new = x_test[200]
print(y_test[200])

prediction = Loded_model.predict(vectorizer.transform([x_new]))
print(prediction)
print(x_new)

if (prediction[0]==0):
  print('The news is Negative Tweet')

else:
  print('The news is Positive Tweet')

1
[1]
love sucki day get unexpect text put smile face
The news is Positive Tweet


In [52]:
x_new = x_test[3]
print(y_test[3])

prediction = Loded_model.predict(vectorizer.transform([x_new]))
print(prediction)
print(x_new)

if (prediction[0]==0):
  print('The news is Negative Tweet')

else:
  print('The news is Positive Tweet')

0
[1]
great etru tonight realli enjoy get plan finish tomorrow
The news is Positive Tweet


In [53]:
x_new = x_test[5]
print(y_test[5])

prediction = Loded_model.predict(vectorizer.transform([x_new]))
print(prediction)
print(x_new)

if (prediction[0]==0):
  print('The news is Negative Tweet')

else:
  print('The news is Positive Tweet')

0
[0]
aw sun gone
The news is Negative Tweet


In [54]:
x_new = x_test[18]
print(y_test[18])

prediction = Loded_model.predict(vectorizer.transform([x_new]))
print(prediction)
print(x_new)

if (prediction[0]==0):
  print('The news is Negative Tweet')

else:
  print('The news is Positive Tweet')

1
[1]
wife spent day tri get hair look good could make happi
The news is Positive Tweet


In [55]:
x_new = x_test[35]
print(y_test[35])

prediction = Loded_model.predict(vectorizer.transform([x_new]))
print(prediction)
print(x_new)

if (prediction[0]==0):
  print('The news is Negative Tweet')

else:
  print('The news is Positive Tweet')

1
[1]
matthardybrand hope day get better
The news is Positive Tweet
